In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

import re
import math
import random
import time

from pymongo import MongoClient

In [2]:
# 设置请求头
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'www.woshipm.com',
    'Referer': 'http://www.woshipm.com/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
def getArticleContent(url, headers):
    """ Get Article Content
    @ param url: str
    @ param headers: dict, request headers
    @ return soup: bs4.BeautifulSoup
    """
    res = requests.post(url=url, headers=headers)  
    res.raise_for_status()  
    res.encoding = 'utf-8'
    
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

In [4]:
# 创建连接
client = MongoClient('localhost', 27017)
# 获取数据库
db = client['iSpider']
# 获取集合
stream = db.woshipm_stream_list

In [5]:
url = stream.find({'author.name':'苏格兰折耳喵'})[0]['permalink']

In [6]:
soup = getArticleContent(url, headers)

In [7]:
# 页面主体内容
article = soup.find('article', class_='article-wrapper')

In [17]:
# 文章标题
title = article.find('header', 'article-header').h2.get_text()

In [18]:
# 文章内容
content = article.find('div', 'grap').get_text()

In [39]:
# 文章标签
tag_list = article.find('div', 'taglist').find_all('a')

In [40]:
content

'\n本文作者将使用多种文本挖掘方法，来分析《全唐诗》。篇幅略长，请耐心阅读^_^\n\n楔子\n近些年来，弘扬中华传统文化的现象级综艺节目不断涌现，如《中国汉字听写大会》、《中国成语大会》、《中国谜语大会》、《中国诗词大会》等，其背后的社会成因，在于人们对中国文化中最精致文字的膜拜心理，虽然浸淫于层出不穷的网络语汇，时时面临“语言荒漠”的窘境，仍心向往之。\n上述节目中，笔者最感兴趣的还是《中国诗词大会》—通过对诗词知识的比拼及赏析，带动全民重温那些曾经学过的古诗词，分享诗词之美，感受诗词之趣，从古人的智慧和情怀中汲取营养，涵养心灵。\n由于在新浪微舆情从事的是语义分析产品方面的工作，平时用到很多文本挖掘的方法。所以，笔者想从文本（数据）挖掘的角度去“探索”全唐诗，挑战一些不同场景下（现代汉语和古汉语）文本处理和分析的异同点，锤炼自己的分析技能；但更想做的是，结合数据之美和诗歌之雅，用跨界思维去发现一些有趣的东西。\n在这里，笔者分析的语料是《全唐诗》，它编校于清康熙四十四年（1705年），得诗四万八千九百余首。\n接下来，笔者将使用多种文本挖掘方法，来分析《全唐诗》。\n以下是本文的行文脉络：\n\n0 文本预处理\n对于古汉语（文言文），尤其是诗词的分词处理可不简单，因为单字词占古汉语词汇统计信息的80％以上，再加上古汉语微言大义，字字千钧，所以针对现代汉语的分词技术往往不适用于它。鉴于此种情况，笔者采取的是逐字切分的处理方式，同时去掉一些常见的虚词，如“之”、“乎”、“者”、“也”。分词和去停用词处理如下所示：\n\n\n经过文本预处理后，就可以进行文本挖掘中最常规的分析—字频统计，看看《全唐诗》中出现最多的字有哪些。\n1 字频分析：唐诗常用高频字分析\n1.1 全局高频字\n首先，让我们来看看去掉这些虚词之后的全局高频字有哪些，笔者这里展示的是TOP148。“人”字排行第一，这体现了《说文解字》里所讲的“人，天地之性最贵者也”，说明唐诗很好的秉承了“以人为本”的中华文化。而后续的“山”、“风”、“月”、“日”、“天”、“云”、“春”等都是在写景的诗句里经常出现的意象。\n\n\n1.2 典型意象分析\n所谓“意象”，就是客观物象经过创作主体独特的情感活动而创造出来的一种艺术形象。简单地说，意象就是寓“意”之“象”，就是用来寄托主观情思的客观物象。在比较文